# Data Pre-Processing and Feature Calculation

Purpose of the notebook: 
* Load in processed EDF data and annotated data labels 
* Check temporal alignment of datasets
* Calculate derived signals such as heart rate
* Calculate features:
  1. EEG features: 
    * to implement first (minimal list):
      * Delta spectral power
      * Zero crossings
      * Abs total power
      * Relative Power in Main Frequency Bands** (for EEG and EOG only): Assessing power distribution across standard EEG frequency bands. 
      * Aperiodicity?
   * Potential additional features:
      * **Standard Deviation:** Measuring the variation in the EEG signal.
      * **Interquartile Range:** Highlighting the spread of the EEG data.
      * **Skewness and Kurtosis:** Assessing the asymmetry and tailedness of the EEG signal distribution.
      * **Hjorth Mobility and Complexity:** Calculating the frequency and complexity of the signal.
      * **Absolute Total Power** in the 0.4-30 Hz Band: Measuring the overall signal power within this frequency range.
      * **Relative Power in Main Frequency Bands** (for EEG and EOG only): Assessing power distribution across standard EEG frequency bands. 
      * **Power Ratios** (e.g., Delta/Beta): Comparing the power in different frequency bands.
          * This is especially important for generalizing to other species
      * **Permutation Entropy:** Estimating the complexity of the signal.
      * **Higuchi and Petrosian Fractal Dimension:** Analyzing the fractal characteristics of the EEG signal.
  2. Heart rate (calculated from ECG) features: 
    * to implement first (minimal list):
      * **Mean:** mean HR across epoch
      * **Standard deviation:** SD of HR across epoch
      * **VLF Power (0-0.001 Hz HR variability)**: Very low frequency (0-0.001 Hz) power
      * **SD of VLF Power:** SD of VLF power across epoch
    * Potential additional features:
      * Time-Domain Features:
        * **RR Intervals and their Variations:** The basic measure of HRV, representing the time intervals between successive heartbeats.
        * **SDNN (Standard Deviation of NN Intervals):** Measures the variability in heart rate.
        * **RMSSD (Root Mean Square of the Successive Differences):** Reflects the short-term components of HRV, particularly influenced by parasympathetic activity.
        * **NN50 and pNN50:** NN50 counts the number of pairs of successive NN intervals that differ by more than 50 ms, and pNN50 is the proportion of NN50 divided by the total number of NN intervals.
      * Frequency-Domain Features:
        * **Power in Different Frequency Bands:** Typically, the power in the Low Frequency (LF, 0.04–0.15 Hz) and High Frequency (HF, 0.15–0.4 Hz) bands are used. LF reflects both sympathetic and parasympathetic activity, while HF is associated with parasympathetic activity.
        * **LF/HF Ratio:** Represents the balance between sympathetic and parasympathetic activity.
      * Nonlinear Features:
        * **Sample Entropy:** Measures the complexity or irregularity of the RR interval time series.
        * **Poincaré Plot Parameters:** Such as SD1 (standard deviation of points perpendicular to the line of identity) and SD2 (standard deviation of points along the line of identity), reflecting short-term and long-term HRV respectively.
      * Geometrical Features:
        * **HRV Triangular Index:** Measures the total number of all NN intervals divided by the height of the histogram of all NN intervals.
        * **TINN (Triangular Interpolation of NN Interval Histogram):** Reflects the baseline width of the RR interval distribution.
      * Statistical and Miscellaneous Features:
        * **Skewness and Kurtosis of RR Intervals:** Indicating the asymmetry and tailedness of the RR interval distribution.
        * **Mean/SD Heart Rate:** Average rate of heartbeats per minute.

## Load data and dependencies

### Load dependencies

# [Data loading](#data_loading)
## [ECG Processing](#ecg_processing)
# [Feature extraction](#feature_extraction)
## [EEG Features](#feature_eeg)
## [ECG Features](#feature_ecg)

### [Frequency Band Exploration](#freq_bands)

In [ ]:
import yasa
import mne
import os
import sys
import scipy
import glob
import six
import wfdb
import pytz
import sklearn
import pomegranate
import pyedflib
import sleepecg
import datetime
import wfdb.processing
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
#import entropy as ent
import seaborn as sns
from matplotlib import mlab as mlab
from sleepecg import detect_heartbeats
import matplotlib.gridspec as gs


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the src directory to the path
current_path = os.getcwd()
src_path = os.path.abspath(os.path.join(current_path, '..', 'src'))
sys.path.insert(0, src_path) 
from feature_extraction import *

<a id='data_loading'></a>

---
# Load in data

---

### Navigate to data folder

In [ ]:
# Construct the relative path to the folder containing processed data
data_folder_path = os.path.abspath(os.path.join("..", "data"))
process_data_path = os.path.abspath(os.path.join("..", "data", "01_processed-data"))
print(process_data_path)
# Get the current working directory
current_path = os.getcwd()

# Check if the current directory ends with the "Data" folder
# if not current_path.endswith("01_processed-data"):
#     # Change the current working directory to the "Data" folder, if not already there
#     os.chdir(process_data_path)
#     print(f"Changed directory to: {os.getcwd()}")
# else:
#     print("Already in the correct data directory.")

# Print the current working directory
print("Current Working Directory:", os.getcwd())

### Read in header information

In [ ]:
# Read the header information to identify channels and their sampling frequencies
info = mne.io.read_raw_edf(f'{process_data_path}/test12_Wednesday_05_ALL_PROCESSED.edf',
                           preload=False).info

# Print the channel information
print(info)

# Identify channels and their corresponding sampling frequencies
channels_info = info['chs']
sampling_freq_map = {}

#### Load in raw data for just day one of the data

In [ ]:
# Load the EDF file, excluding the EOGs and EKG channels
raw = mne.io.read_raw_edf(f'{process_data_path}/test12_Wednesday_05_DAY1_PROCESSED.edf', preload=True)
# raw.resample(100)                      # Downsample the data to 100 Hz
# raw.filter(0.1, 40)                    # Apply a bandpass filter from 0.1 to 40 Hz
# raw.pick_channels(['C4-A1', 'C3-A2'])  # Select a subset of EEG channels
raw # Outputs summary data about file

# Inspect Data
print(raw.info)
print('The channels are:', raw.ch_names)
print('The sampling frequency is:', raw.info['sfreq'])

# Rename channels (replace spaces if any)
channel_renaming_dict = {name: name.replace(' ', '_') for name in raw.ch_names}
raw.rename_channels(channel_renaming_dict)
print('The channels are:', raw.ch_names)

# ['ECG_Raw_Ch1', 'ECG_ICA2', 'LEOG_Pruned_Ch2', 'LEMG_Pruned_Ch4', 'REEG2_Pruned_Ch7', 'LEEG3_Pruned_Ch8', 
# 'REEG2_Raw_Ch7', 'LEEG3_Raw_Ch8', 'EEG_ICA5', 'pitch', 'roll', 'heading', 'GyrZ', 'MagZ', 'ODBA', 'Pressure']

# Assuming 'raw' is your Raw object from MNE
channel_types = {}

for ch in raw.ch_names:
    if ch.startswith('ECG'):
        channel_types[ch] = 'ecg'
    elif ch.startswith(('LEOG', 'REOG')):
        channel_types[ch] = 'eog'
    elif ch.startswith(('LEMG', 'REMG')):
        channel_types[ch] = 'emg'
    elif ch.startswith(('LEEG', 'REEG')):
        channel_types[ch] = 'eeg'
    elif ch in ['pitch', 'roll', 'heading']:
        channel_types[ch] = 'resp'
    elif ch in ['GyrZ', 'MagZ', 'ODBA']:
        channel_types[ch] = 'syst'
    elif ch in ['Pressure']:
        channel_types[ch] = 'misc'
    elif ch == 'Heart_Rate':
        channel_types[ch] = 'bio'

# Now set the channel types
raw.set_channel_types(channel_types)
print()

#### Get metadata from file

Get start time, channel names, etc.

In [ ]:
channels = ['ECG_Raw_Ch1', 'ECG_ICA2', 'LEOG_Pruned_Ch2', 'LEMG_Pruned_Ch4', 'REEG2_Pruned_Ch7',
            'LEEG3_Pruned_Ch8', 'REEG2_Raw_Ch7', 'LEEG3_Raw_Ch8', 'EEG_ICA5', 'pitch', 'roll',
            'heading', 'GyrZ', 'MagZ', 'ODBA', 'Pressure']
#     sns.kdeplot(raw.copy().pick([channel]).get_data()[0,:]).set_title(channel)
#     plt.show()

In [ ]:
# Inspect Data
print(raw.info)
print('The channels are:', raw.ch_names)
print('The sampling frequency is:', raw.info['sfreq'])

# Extract the measurement date (start time) from raw.info
start_time = raw.info['meas_date']
fs = raw.info['sfreq']

# Define the PST timezone
pst_timezone = pytz.timezone('America/Los_Angeles')

# Convert to datetime object in PST
if isinstance(start_time, datetime.datetime):
    # If it's already a datetime object, just replace the timezone
    recording_start_datetime = start_time.replace(tzinfo=None).astimezone(pst_timezone)
elif isinstance(start_time, (int, float)):
    # Convert timestamp to datetime in PST
    recording_start_datetime = pst_timezone.localize(datetime.datetime.fromtimestamp(start_time))
else:
    # Handle other formats if necessary
    pass

# Calculate the recording duration in seconds
recording_duration_seconds = len(raw) / fs

# Calculate the recording end datetime
recording_end_datetime = recording_start_datetime + datetime.timedelta(seconds=recording_duration_seconds)

# Calculate duration as a timedelta object
duration_timedelta = datetime.timedelta(seconds=recording_duration_seconds)

# Create a time index
#time_index = pd.date_range(recoring_start_datetime, recording_end_datetime)

# Format duration into days, hours, minutes, and seconds
days = duration_timedelta.days
hours, remainder = divmod(duration_timedelta.seconds, 3600)
minutes, seconds = divmod(remainder, 60)

print('The start time in PST (Los Angeles) is:', recording_start_datetime)
print('The end time in PST (Los Angeles) is:', recording_end_datetime)
print(f'Duration: {days} days, {hours} hours, {minutes} minutes, {seconds} seconds')


<a id='ecg_processing'></a>

---
## ECG Pre-Processing

---

Performing peak-detection and corrections with `sleepecg` package. 

Many thanks to Sam Proell's helpful tutorial on the topic:
https://www.samproell.io/posts/signal/ecg-library-comparison/

### Step 1: Select ECG Channel for Pre-Processing

In [ ]:
# Find all channel names that contain "ECG"
ecg_channels = [ch for ch in raw.info['ch_names'] if 'ECG' in ch]

# Help user define which should be used by visualizing each to find the better channel
# Define the duration to plot (in seconds)
plot_duration = 30

# Use the start and end index found in the section above

start_index = int(2000 * fs) # halfway throughout the day
end_index = start_index + int(plot_duration * fs)

# Create a figure
fig = go.Figure()

# Offset amount
offset = 0.005
current_offset = 0  # Start with no offset for the first channel

# Plot the first 30 seconds of all ECG channels
for idx, channel in enumerate(ecg_channels):
    ecg_data = raw.copy().pick([channel]).get_data()[0, start_index:int(start_index + plot_duration*fs)]
    time_vector = raw.times[start_index:end_index]
    
    # Offset the ECG data for visualization
    ecg_data_with_offset = ecg_data + (idx * current_offset)
    
    # Add the ECG data trace for each channel
    fig.add_trace(go.Scatter(x=time_vector, y=ecg_data_with_offset, mode='lines', 
                             name= f'{idx} - {channel}'))

    # Increase the offset for the next channel
    current_offset += offset

# Update layout for better readability
fig.update_layout(title='ECG Channel Comparison',
                  xaxis_title='Time (seconds)',
                  yaxis_title='Amplitude',
                  showlegend=True)

# Show the figure
fig.show()

### Step 2: Perform peak-detection

Use the channel index in the legend above to select a channel to use for peak detection.

In [ ]:
# Peak-detection

# Manually set the index of the channel you want to use for peak detection
# INPUT HERE:
selected_channel_index = 0  # Replace 0 with the index of the channel you want to use

# Now use this index to extract the data for peak detection
selected_channel = ecg_channels[selected_channel_index]
ecg_data = raw.copy().pick([selected_channel]).get_data()[0]

rpeaks = detect_heartbeats(ecg_data, fs) # using sleepecg
print(rpeaks[0:10])
print(fs)

print('Peak detection ran successfully.')

rpeaks_corrected = wfdb.processing.correct_peaks(
    ecg_data, rpeaks, search_radius=200, smooth_window_size=50, peak_dir="up"
)
# MIGHT HAVE TO UPDATE search_radius
wfdb.plot_items(ecg_data, [rpeaks_corrected])  # styling options omitted


### Visualizing Peak Detection Results

Generating a plot at a given timestamp to visualize the results of the ECG peak-detection method.

In [ ]:
# Start string and parsing
start_str = "10/25/2019 20:30:30"
start_time = datetime.datetime.strptime(start_str, '%m/%d/%Y %H:%M:%S')
start_time = pst_timezone.localize(start_time)
time_diff = (start_time - recording_start_datetime).total_seconds()

duration = 2000 # 33 minutes..? gonna make this a little longer
duration = 7200 # 2 hours
fs = raw.info['sfreq']  # Sampling frequency

# Calculate the starting index
start_index = int(fs * time_diff)

# Extract a segment starting from start_index
end_index = start_index + int(duration * fs)
print(start_index, end_index)
ecg_segment = ecg_data[start_index:end_index]

# Create a time vector for the segment
segment_time = [i / fs for i in range(end_index - start_index)]

# Calculate R-peaks within the segment's range
rpeaks_segment = [rp for rp in rpeaks if start_index <= rp < end_index]
rpeaks_segment_time = [(rp - start_index) / fs for rp in rpeaks_segment]
rpeaks_segment_amplitudes = [ecg_segment[rp - start_index] for rp in rpeaks_segment]

# Calculate the time points for the corrected R-peaks within the segment
rpeaks_corrected_segment = [rp for rp in rpeaks_corrected if start_index <= rp < end_index]
rpeaks_corrected_segment_time = [(rp - start_index) / fs for rp in rpeaks_corrected_segment]
rpeaks_corrected_amplitudes = [ecg_segment[rp - start_index] for rp in rpeaks_corrected_segment]

# Calculate heart rates between R-peaks
heart_rates = [60 / ((rpeaks_segment[i+1] - rpeaks_segment[i]) / fs) for i in range(len(rpeaks_segment) - 1)]
# Create a heart rate array matching the frequency of the ECG trace
hr_data = np.zeros_like(ecg_segment)
# Assign heart rate values to the intervals between R-peaks
for i in range(len(rpeaks_segment) - 1):
    start_idx = rpeaks_segment[i] - start_index
    end_idx = rpeaks_segment[i+1] - start_index
    hr_data[start_idx:end_idx] = heart_rates[i]

# Calculate heart rates between R-peaks
heart_rates2 = [60 / ((rpeaks_corrected_segment[i+1] - rpeaks_corrected_segment[i]) / fs) for i in range(len(
    rpeaks_corrected_segment) - 1)]
# Create a heart rate array matching the frequency of the ECG trace
hr_data2 = np.zeros_like(ecg_segment)
# Assign heart rate values to the intervals between R-peaks
for i in range(len(rpeaks_corrected_segment) - 1):
    start_idx = rpeaks_corrected_segment[i] - start_index
    end_idx = rpeaks_corrected_segment[i+1] - start_index
    hr_data2[start_idx:end_idx] = heart_rates2[i]

# Create subplots with shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Add ECG data trace
fig.add_trace(go.Scatter(x=segment_time, y=ecg_segment, mode='lines', name='ECG'), row=1, col=1)

# Add original R-peaks as red scatter points
fig.add_trace(go.Scatter(x=rpeaks_segment_time, y=rpeaks_segment_amplitudes, mode='markers', name='R-peaks',
                         marker=dict(size=10, color='red')), row=1, col=1)

# Add corrected R-peaks as orange scatter points
fig.add_trace(go.Scatter(x=rpeaks_corrected_segment_time, y=rpeaks_corrected_amplitudes, mode='markers',
                         name='Corrected R-peaks', marker=dict(size=10, color='orange')), row=1, col=1)

# Add heart rate trace
fig.add_trace(go.Scatter(x=segment_time, y=hr_data, mode='lines', name='Heart Rate', line=dict(color='red')),
              row=2, col=1)

# Add heart rate trace
fig.add_trace(go.Scatter(x=segment_time, y=hr_data2, mode='lines', name='Heart Rate', line=dict(color='blue')),
              row=2, col=1)

# Update layout for better readability
fig.update_layout(title=f'ECG Data and Heart Rate ({duration} seconds)',
                  xaxis_title='Time (seconds)',
                  yaxis_title='Amplitude / Heart Rate',
                  showlegend=True)

# Show the figure
fig.show()


In [ ]:
# less fancy: plt.plot(ecg_signal); plt.plot(rpeaks, ecg_signal[rpeaks], "x")
fig = wfdb.plot_items(
    ecg_data,
    [rpeaks],
    fs=fs,
    sig_name=["ECG"],
    sig_units=["mV"],
    time_units="seconds",
    return_fig=True,
    ann_style="o",
)

In [ ]:
# Calculate heart rates between R-peaks
heart_rates = [60 / ((rpeaks_segment[i+1] - rpeaks_segment[i]) / fs) for i in range(len(rpeaks_segment) - 1)]

# Create a heart rate array matching the frequency of the ECG trace
hr_data = np.zeros_like(ecg_segment)

# Assign heart rate values to the intervals between R-peaks
for i in range(len(rpeaks_segment) - 1):
    start_idx = rpeaks_segment[i] - start_index
    end_idx = rpeaks_segment[i+1] - start_index
    hr_data[start_idx:end_idx] = heart_rates[i]

# Create subplots with shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Add ECG data trace
fig.add_trace(go.Scatter(x=segment_time, y=ecg_segment, mode='lines', name='ECG'), row=1, col=1)
# Add R-peaks as scatter points
fig.add_trace(go.Scatter(x=rpeaks_segment_time, y=rpeaks_segment_amplitudes, mode='markers', name='R-peaks',
                         marker=dict(size=10, color='red')), row=1, col=1)

# Add heart rate trace
fig.add_trace(go.Scatter(x=segment_time, y=hr_data, mode='lines', name='Heart Rate', line=dict(color='blue')),
              row=2, col=1)

# Update layout for better readability
fig.update_layout(title=f'ECG Data and Heart Rate ({duration} seconds)',
                  xaxis_title='Time (seconds)',
                  yaxis_title='Amplitude / Heart Rate',
                  showlegend=True)

# Show the figure
fig.show()


<a id='feature_extraction'></a>

---
# Feature extraction
---

1. EEG features: 
* to implement first (minimal list):
  * Delta spectral power
  * Zero crossings
  * Abs total power
2. Heart rate (calculated from ECG) features: 
* to implement first (minimal list):
  * **Mean:** mean HR across epoch
  * **Standard deviation:** SD of HR across epoch
  * **VLF Power (0-0.001 Hz HR variability)**: Very low frequency (0-0.001 Hz) power
  * **SD of VLF Power:** SD of VLF power across epoch

<a id='feature_eeg'></a>
## EEG

### Delta spectral power

Good resource for spectral decomp:<br>
https://github.com/pennmem/PythonBootcamp/blob/master/Day%204%20-%20Spectral%20Decomp.ipynb<br>
How fourier transforms work:<br>
https://neuraldatascience.io/7-eeg/time_freq.html

In [ ]:
from IPython.display import Image 
from IPython.core.display import HTML 

img_path = '../helpful-figs/' 
display(Image(filename = img_path + 'slow-wave-1.png', width=800, height=300))
display(Image(filename = img_path + 'slow-wave-2.png', width=800, height=300))
display(Image(filename = img_path + 'rem.png', width=800, height=300))


In [ ]:
ecg_data = raw.copy().pick(['ECG_Raw_Ch1']).get_data()[0]
eeg_data = raw.copy().pick(['EEG_ICA5']).get_data()[0]
# If you loaded in the full Wednesday file and not just day 1:
# one_day = 500 * 60 * 60 * 24 # 500 data points per second, 60 sec/min, 60 min/hr, 24 hr/day
# ecg_data = raw.copy().pick(['ECG_ICA2']).get_data()[0, :one_day]
# eeg_data = raw.copy().pick(['EEG_ICA5']).get_data()[0, :one_day]


In [ ]:
print(len(ecg_data), len(eeg_data), sep='\n')

In [ ]:
delta_power = get_rolling_band_power_welch(eeg_data, 0, len(eeg_data), freq_range=(0.5, 4), ref_power=1,
                                                freq=500, window_sec=30, step_size=1)
sns.kdeplot(delta_power) # TODO explore: there are some crazy low negative numbers
plt.show()

### Zero crossings

In [ ]:
zero_crossings = get_rolling_zero_crossings(eeg_data, 0, len(eeg_data))
sns.kdeplot(zero_crossings)
plt.show()

### Absolute total power (and power of each frequency band)
(this cell takes a few minutes)

In [ ]:
FREQ_BANDS = {
    "delta": (0.4, 4.0),
    "theta": (4.0, 8.5),
    "alpha": (8.5, 11.5),
    "sigma": (11.5, 15.5),
    "beta": (15.5, 30)
}
freq_band_names = list(FREQ_BANDS.keys())
freq_band_powers = {}
for freq_band in FREQ_BANDS.keys():
    print(freq_band)
    band_power = get_rolling_band_power_welch(eeg_data, 0, len(eeg_data), freq_range=FREQ_BANDS[freq_band],
                                              ref_power=1, freq=500, window_sec=2, step_size=1)
    freq_band_powers[freq_band] = band_power

In [ ]:
abs_power_dB = get_rolling_band_power_welch(eeg_data, 0, len(eeg_data), freq_range=(0.4, 30),
                                            ref_power=1, freq=500, window_sec=2, step_size=1)
sns.kdeplot(abs_power_dB) # TODO explore: Has some crazy low values
plt.show()

<a id='feature_ecg'></a>

## Heart rate (from ECG)

### Rolling mean & standard deviation

In [ ]:
hr_data = get_heart_rate(ecg_data)

In [ ]:
hr_mean, hr_std = get_rolling_mean_std(hr_data, 0, len(hr_data))
sns.kdeplot(hr_mean)

In [ ]:
sns.kdeplot(hr_std)

In [ ]:
sns.kdeplot(hr_data) # TODO explore: something is weird about this why is there a value at 30000
# DONE - cleaned up heart rate code to fill in wacky outliers with the average of its neighbors

### VLF Power

In [ ]:
vlf_power = get_rolling_band_power_fourier_sum(ecg_data, 0, len(ecg_data), freq_range=(0, 0.01), window_sec=20,
                                         ref_power=1)
sns.kdeplot(vlf_power[vlf_power < np.quantile(pd.Series(vlf_power).dropna(), 0.98)]) # drop outliers
plt.show()

USE 8000 for window

### SD of VLF Power

In [ ]:
_, vlf_power_std = get_rolling_mean_std(vlf_power, 0, len(vlf_power), window_sec=30)
sns.kdeplot(vlf_power_std)
plt.show()

<a id='freq_bands'></a>

# Freq bands

In [ ]:
# Load labeled data
# Path to CSV with scored data
file_path = f'{data_folder_path}/02_annotated-data/test12_Wednesday_06_Hypnogram_JKB_1Hz.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)
df['R.Time'] = pd.to_datetime(df['R.Time']).dt.tz_localize('America/Los_Angeles')
df['Sleep.Code'].value_counts(normalize=True)

In [ ]:
# start_time and end_time obtained from LabChart as this section goes through most of the stages of wake/sleep
start_time = datetime.datetime(2019, 10, 26, hour=5, minute=30, second=30, tzinfo=pytz.timezone('America/Los_Angeles'))
end_time = datetime.datetime(2019, 10, 26, hour=5, minute=45, second=0, tzinfo=pytz.timezone('America/Los_Angeles'))
duration = (start_time - end_time).seconds
duration_interval = duration * 500
start_index = (start_time - recording_start_datetime).seconds * 500
end_index = (end_time - recording_start_datetime).seconds * 500

labels_start_idx =  df[df['R.Time'] == start_time].index[0]
labels_end_idx = df[df['R.Time'] == end_time].index[0]

df_test = df.loc[labels_start_idx:labels_end_idx-1]

eeg_subset = eeg_data[start_index:end_index]
sw1_filter = df_test['Sleep.Num'] == 4
sw2_filter = df_test['Sleep.Num'] == 5
# sw2_filter = np.array([[x] * 500 for x in sw2_filter]).flatten()
rem_filter = df_test['Sleep.Num'] == 7
# rem_filter = np.array([[x] * 500 for x in rem_filter]).flatten()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate sample EEG data (replace this with your actual EEG data)
# Sample EEG data is often represented as a 1D array of voltage values over time
# For demonstration purposes, we'll generate a sine wave signal
sampling_freq = 500  # Sample frequency in Hz
t = np.linspace(0, duration, int(sampling_freq * duration))

# Perform FFT
fft_result = np.fft.fft(eeg_subset)
freqs = np.fft.fftfreq(len(fft_result), 1 / sampling_freq)

# Keep only positive frequencies (we're dealing with real-valued signals)
positive_freqs = freqs[:len(freqs)//2]
power_spectrum = np.abs(fft_result[:len(fft_result)//2]) ** 2

# Plot frequency domain power distribution
plt.figure(figsize=(10, 6))
index_of_20hz = np.where(positive_freqs > 20)[0][0]
plt.plot(positive_freqs[:index_of_20hz], power_spectrum[:index_of_20hz])
plt.ylim([0, 0.1])
plt.title('Frequency Domain Power Distribution')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.grid(True)
plt.show()
